In [1]:
%config Completer.use_jedi = False

In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import statistics
import math
from datetime import datetime
from datetime import timedelta
from dateutil import relativedelta
import glob
import warnings

warnings.filterwarnings('ignore')
pd.options.mode.chained_assignment = None

In [3]:
# Caminhos dos diretórios raiz                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                                        
# general_path = 'C:/Users/cnalm/OneDrive/Hidroweb'    # Caminho do projeto (Cristiano)
# general_path = 'D:/Dados_Nuvem/OneDrive/Hidroweb'   # Colocar aqui o caminho de Filipe
general_path = r'C:/Users/linde/OneDrive/Hidroweb'   # Colocar aqui o caminho de Lindemberg

states = ['PARÁ', 'AMAPÁ', 'RORAIMA', 'AMAZONAS', 'MARANHÃO', 'CEARÁ', 'PIAUÍ', 'PERNAMBUCO', 
          'RIO GRANDE DO NORTE', 'TOCANTINS', 'PARAÍBA', 'ACRE', 'ALAGOAS', 'BAHIA', 'MATO GROSSO',
          'RONDÔNIA', 'SERGIPE', 'GOIÁS', 'MINAS GERAIS', 'DISTRITO FEDERAL', 'MATO GROSSO DO SUL',
          'ESPÍRITO SANTO', 'SÃO PAULO', 'RIO DE JANEIRO', 'PARANÁ', 'SANTA CATARINA', 'RIO GRANDE DO SUL']

### Reading hdf files ###

In [4]:
df_filter_pr = pd.read_hdf(general_path + r'/Consolidated Files/BRASIL_RAW_1855_2022_PR450.h5')
df_filter_pr.tail(5)

,Date,Value,Code,Year
128939050,1996-02-25,25.8,08560000,1996
128939051,1996-02-26,0.0,08560000,1996
128939052,1996-02-27,0.0,08560000,1996
128939053,1996-02-28,0.0,08560000,1996
128939054,1996-02-29,0.0,08560000,1996


In [5]:
df_gauges = pd.read_hdf(general_path + r'/Consolidated Files/BRASIL_RAW_NOT_CEMADEN_LIST_GAUGES.h5')
df_gauges = df_gauges[["Code", "State"]]
df_gauges.tail(5)

,Code,State
11721,08460003,RORAIMA
11722,08460004,RORAIMA
11723,08461000,RORAIMA
11724,08464001,RORAIMA
11725,08560000,RORAIMA


In [6]:
df_filter_pr_state = df_filter_pr.merge(df_gauges, how = 'left', on = 'Code')
del df_filter_pr
del df_gauges
df_filter_pr_state.tail(5)

,Date,Value,Code,Year,State
128939050,1996-02-25,25.8,08560000,1996,RORAIMA
128939051,1996-02-26,0.0,08560000,1996,RORAIMA
128939052,1996-02-27,0.0,08560000,1996,RORAIMA
128939053,1996-02-28,0.0,08560000,1996,RORAIMA
128939054,1996-02-29,0.0,08560000,1996,RORAIMA


### FUNCTION

In [7]:
def calculateP(state):
    df_filter_pr_state_filter = df_filter_pr_state[(df_filter_pr_state['State'] == state)]
    station_list = df_filter_pr_state_filter['Code'].drop_duplicates().reset_index(drop = True, inplace = False).values.tolist()
    df_q_outlier = pd.DataFrame(columns = ["Code", "Date", "Outlier"])
    count = 0
    print(state)
    for station in station_list:
        count = count + 1
        df_temp_station = df_filter_pr_state_filter[(df_filter_pr_state_filter['Code'] == station)]
        df_temp_station['Month'] = df_temp_station['Date'].dt.month
        df_threshold = df_temp_station[df_temp_station['Value']>= 1.0]
        df_threshold = df_threshold.reset_index(drop = True, inplace=False)
        df_month = pd.DataFrame(df_threshold['Month'].drop_duplicates().reset_index(drop = True, inplace = False).values.tolist(), columns=['Month'])
        df_month = df_month.sort_values(['Month'], ignore_index=True)
        list_month = df_month['Month'].drop_duplicates().reset_index(drop = True, inplace = False).values.tolist()
        upper_whisker_list = []
        for i in list_month:
            df_threshold_temp = df_threshold[(df_threshold['Month'] == i)]
        #     print(df_threshold_temp)
            first_quartile = df_threshold_temp['Value'].quantile(0.25)
            thrid_quartile = df_threshold_temp['Value'].quantile(0.75)
            iqr = thrid_quartile - first_quartile
            upper_whisker = thrid_quartile + 1.5 * iqr
            upper_whisker_list.append(upper_whisker)
#             print('iqr', iqr,"month", i)
#             print('thrid_quartile', thrid_quartile,"month", i)
#             print('upper_whisker', upper_whisker,"month", i)
#         print('upper_whisker_list',upper_whisker_list)
        df_quartile_threshold = pd.DataFrame(list(zip(list_month, upper_whisker_list)), columns =['Month', 'Upper Whisker'])
        print(count, '/', len(station_list))
        year_list = df_temp_station['Year'].drop_duplicates().reset_index(drop = True, inplace = False).values.tolist()
#         print(year_list)
        list_code = []
        list_date = []
        list_outlier= []
        for year in year_list:
            ### FILTERING BY YEAR ###
            df_temp_station_year = df_temp_station[(df_temp_station['Year'] == year)]
            df_temp_station_year['Month'] = df_temp_station_year['Date'].dt.month
            list_month = df_temp_station_year['Month'].drop_duplicates().reset_index(drop = True, inplace = False).values.tolist()
            for month in list_month:
                df_temp_station_month = df_temp_station_year[(df_temp_station_year['Month'] == month)]
                count_ops = len(df_temp_station_month)
                min_date = min(df_temp_station_month['Date'])
                max_date = max(df_temp_station_month['Date'])
                first_day = datetime(year, month, 1)
                last_day = first_day + relativedelta.relativedelta(months=1)
                date_list = []
                today = first_day
                while today < last_day:
                    date_list.append(today)
                    today = today + timedelta(days=1)
                df_date_list = pd.DataFrame(date_list, columns=['Date'])
                df_analysis = df_date_list.merge(df_temp_station_month, how = 'left', on  = 'Date')
                df_analysis = df_analysis.sort_values(['Date'], ignore_index=True)
                len_period = len(df_analysis)
                df_merge = df_analysis.merge(df_quartile_threshold, how="left", on ='Month')
                df_merge['Validation Count'] = np.where(df_merge['Value'] > df_merge['Upper Whisker'], 0, 1)
                upper_validation = df_merge['Validation Count'].sum()
                q_outlier = 100.0 * upper_validation / len_period
    #             print(q_outlier)
                list_code.append(station)
                list_date.append(first_day)
                list_outlier.append(q_outlier)
    #     print(list_code)
    #     print(list_date)
        # print(list_outlier)
        df_station_outlier = pd.DataFrame(list(zip(list_code, list_date, list_outlier)),
                                             columns =['Code', 'Date', 'Outlier'])
        df_q_outlier = pd.concat([df_q_outlier, df_station_outlier], ignore_index = True)
        del df_station_outlier
        df_q_outlier['State'] = state
    df_q_outlier.to_hdf(general_path + '/Quality/Q3/Monthly/BRASIL_Q3_OUTLIER_'+state+'_by_month.h5', 'table_data', mode = 'w', append = False, complevel = 9, encoding="cp860")
    data = np.array(df_q_outlier['Outlier'])
    plt.rcParams["figure.figsize"] = (10, 5)
    plt.hist(data, bins = 10, edgecolor='black', color = 'grey') #data and number of bins
    tnr = {'fontname':'Times New Roman', 'fontsize':12}
#     plt.xlim(0, 100)
#     plt.axvline(data.mean(), color='k', linestyle='dashed', linewidth=2)
    plt.tight_layout()
    plt.savefig(general_path+'/Figures/q3_outlier_'+state+'_by_month.jpeg', format='jpeg', dpi=1200, bbox_inches='tight') 
    # plt.show()
    plt.close()
    # print("mean", data.mean())
    # print("median", statistics.median(data))
    # print("mode", statistics.mode(data))
    # print(df_q_outlier.head(10))
    del df_q_outlier
    del data

In [8]:
# calculateP("AMAPÁ")
for state in states:
    calculateP(state)

PARÁ
1 / 285
2 / 285
3 / 285
4 / 285
5 / 285
6 / 285
7 / 285
8 / 285
9 / 285
10 / 285
11 / 285
12 / 285
13 / 285
14 / 285
15 / 285
16 / 285
17 / 285
18 / 285
19 / 285
20 / 285
21 / 285
22 / 285
23 / 285
24 / 285
25 / 285
26 / 285
27 / 285
28 / 285
29 / 285
30 / 285
31 / 285
32 / 285
33 / 285
34 / 285
35 / 285
36 / 285
37 / 285
38 / 285
39 / 285
40 / 285
41 / 285
42 / 285
43 / 285
44 / 285
45 / 285
46 / 285
47 / 285
48 / 285
49 / 285
50 / 285
51 / 285
52 / 285
53 / 285
54 / 285
55 / 285
56 / 285
57 / 285
58 / 285
59 / 285
60 / 285
61 / 285
62 / 285
63 / 285
64 / 285
65 / 285
66 / 285
67 / 285
68 / 285
69 / 285
70 / 285
71 / 285
72 / 285
73 / 285
74 / 285
75 / 285
76 / 285
77 / 285
78 / 285
79 / 285
80 / 285
81 / 285
82 / 285
83 / 285
84 / 285
85 / 285
86 / 285
87 / 285
88 / 285
89 / 285
90 / 285
91 / 285
92 / 285
93 / 285
94 / 285
95 / 285
96 / 285
97 / 285
98 / 285
99 / 285
100 / 285
101 / 285
102 / 285
103 / 285
104 / 285
105 / 285
106 / 285
107 / 285
108 / 285
109 / 285
110 / 285
111

In [9]:
all_files = glob.glob(gen
                      eral_path + '/Quality/Q3/Monthly/*.h5')
# print(all_files)
count = 0
df = []
for filename in all_files:
    df = pd.read_hdf(filename)
    df.reset_index(inplace=True, drop = True)
#     print(df)
    if count == 0:
        df_total = df.copy(deep = True)
    else:
        df_total = pd.concat([df_total, df], ignore_index = True)
    del df
    count = count + 1
    print("File", count,"|", filename)
print(count, "files are loaded")

File 1 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_ACRE_by_month.h5
File 2 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_ALAGOAS_by_month.h5
File 3 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_AMAPÁ_by_month.h5
File 4 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_AMAZONAS_by_month.h5
File 5 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_BAHIA_by_month.h5
File 6 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_CEARÁ_by_month.h5
File 7 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_DISTRITO FEDERAL_by_month.h5
File 8 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_ESPÍRITO SANTO_by_month.h5
File 9 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_GOIÁS_by_month.h5
File 10 | C:/Users/linde/OneDrive/Hidroweb/Quality/Q3/Monthly\BRASIL_Q3_OUTLIER_MARANHÃO_by_month.h5
Fi

In [13]:
df_total.to_hdf(general_path + '/Quality/Monthly/BRASIL_Q3_by_month.h5', 'table_data', mode = 'w', append = False, complevel = 9, encoding="cp860")
df_total.head(5)

,Code,Date,Outlier,State
0,00771001,1982-07-01,100.000000,ACRE
1,00771001,1982-08-01,100.000000,ACRE
2,00771001,1982-09-01,100.000000,ACRE
3,00771001,1982-10-01,96.774194,ACRE
4,00771001,1982-11-01,100.000000,ACRE
